In [55]:
import cv2
import numpy as np
import pandas as pd
from keras import Model
from keras.src.applications import EfficientNetB0
from keras.src.callbacks import EarlyStopping
from keras.src.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from keras.src.optimizers import Adam
from keras.src.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import keras
from sklearn.metrics import classification_report

In [56]:
# One Hot encode the labels

data = "../Data/data_directory.csv"

df = pd.read_csv(data)

# Drop the rows where Grade is I
df = df[df["Grade"] != "I"]

# 0 MEANS IT IS A, 1 MEANS IT IS B, C, or D
df['Grade'] = df['Grade'].apply(lambda x: 0 if x == 'A' else 1)

df.head()

df["Pixels"] = ""


In [57]:
# Turn the images into pixel values

image_folder = "../Data/data"

train_ratio = 0.8
val_ratio = 0.1

for index, row in df.iterrows():
    img_path = image_folder + "/" + row["Image"]
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)
    df.at[index, "Pixels"] = img
df.head()


,Image,Grade,Pixels
0,a_IMAGE_001_left_ankle.jpg,1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,a_IMAGE_001_left_calf.jpg,1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,a_IMAGE_001_left_high_thigh.jpg,1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,a_IMAGE_001_left_low_thigh.jpg,1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,a_IMAGE_001_left_metatarsal.jpg,1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [58]:
# Split the data into train, validation, and test sets

# Drop the Image column
df = df.drop(columns=["Image"])

# Split the data
train, test = train_test_split(df, test_size=0.2, random_state=42)

train, val = train_test_split(train, test_size=0.1, random_state=42)

X_train = np.array(train["Pixels"].tolist())
X_val = np.array(val["Pixels"].tolist())
X_test = np.array(test["Pixels"].tolist())
y_train = np.array(train["Grade"].tolist())
y_val = np.array(val["Grade"].tolist())
y_test = np.array(test["Grade"].tolist())

In [59]:
data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

data_augmentation.fit(X_train)

In [60]:
efficient_net_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(efficient_net_model.output)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=efficient_net_model.input, outputs=predictions)

for layer in efficient_net_model.layers[-20:]:
    layer.trainable = True
    
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

batch_size = 32
epochs = 100

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/100
186/186 [==============================] - 645s 3s/step - loss: 0.4325 - accuracy: 0.8150 - val_loss: 0.5989 - val_accuracy: 0.6616 - lr: 1.0000e-04
Epoch 2/100
186/186 [==============================] - 606s 3s/step - loss: 0.2783 - accuracy: 0.8839 - val_loss: 0.7086 - val_accuracy: 0.6858 - lr: 1.0000e-04
Epoch 3/100
186/186 [==============================] - 586s 3s/step - loss: 0.2184 - accuracy: 0.9116 - val_loss: 0.3818 - val_accuracy: 0.8444 - lr: 1.0000e-04
Epoch 4/100
186/186 [==============================] - 580s 3s/step - loss: 0.1790 - accuracy: 0.9242 - val_loss: 0.4962 - val_accuracy: 0.8338 - lr: 1.0000e-04
Epoch 5/100
186/186 [==============================] - 578s 3s/step - loss: 0.1275 - accuracy: 0.9531 - val_loss: 0.3286 - val_accuracy: 0.8807 - lr: 1.0000e-04
Epoch 6/100
186/186 [==============================] - 612s 3s/step - loss: 0.1101 - accuracy: 0.9561 - val_loss: 0.4495 - val_accuracy: 0.8595 - lr: 1.0000e-04
Epoch 7/100
186/186 [=============

In [61]:
from sklearn.metrics import precision_recall_fscore_support

model_scores = model.evaluate(X_test, y_test)
print("Test Loss:", model_scores[0])
print("Test Accuracy:", model_scores[1])


52/52 [==============================] - 37s 717ms/step - loss: 0.3959 - accuracy: 0.8869
Test Loss: 0.3958946466445923
Test Accuracy: 0.8869407773017883


In [65]:
y_pred = model.predict(X_test)

y_pred_classes = np.where(y_pred > 0.5, 1, 0)

print(classification_report(y_test, y_pred_classes))

52/52 [==============================] - 41s 789ms/step
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       975
           1       0.87      0.86      0.86       679

    accuracy                           0.89      1654
   macro avg       0.88      0.88      0.88      1654
weighted avg       0.89      0.89      0.89      1654


In [67]:
# Save the model
model.save("waveformMedModel.keras")